In [1]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
import yaml
with open("../assets/doml_meta.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

In [2]:
import prettyprinter as pp
from prettyprinter.prettyprinter import IMPLICIT_MODULES
pp.install_extras(include=['dataclasses'])

In [3]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.metamodel')
pp.pprint(mm["application_SoftwareComponent"])

mc_openapi.doml_mc.intermediate_model.metamodel.DOMLClass(
    name='application_SoftwareComponent',
    superclass='application_ApplicationComponent',
    attributes={
        'isPersistent': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='isPersistent',
            type='Boolean',
            multiplicity=('1', '1'),
            default=False
        ),
        'licenseCost': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='licenseCost',
            type='String',
            multiplicity=('0', '1'),
            default=None
        ),
        'configFile': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='configFile',
            type='String',
            multiplicity=('0', '1'),
            default=None
        )
    },
    associations={
        'exposedInterfaces': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAssociation(
            name='exposedInterfaces',
            class_

### Load the DOML document here:

In [4]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
doml_document_path = "../../tests/doml/nginx-openstack_v2.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2_wrong.domlx"
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

doml_model = parse_doml_model(doc, mm)
print(doml_model)

DOMLModel(name='nginx_openstack', description=None, application=Application(name='app', children={'nginx': ApplicationComponent(typeId='application_SoftwareComponent', name='nginx', consumedInterfaces=[], exposedInterfaces=[], attributes={'isPersistent': False})}), infrastructure=Infrastructure(nodes={'vm1': InfrastructureNode(name='vm1', typeId='infrastructure_VirtualMachine', network_interfaces={}, attributes={}, associations={})}, networks={'net1': Network(name='net1', protocol='tcp/ip', addressRange='16.0.0.0/24')}, groups={'sg': Group(name='sg', typeId='infrastructure_SecurityGroup')}), optimization=Optimization(typeId='optimization_OptimizationLayer'), concretizations={'con_infra': Concretization(name='con_infra', groups={}, vms={'concrete_vm': VirtualMachine(name='concrete_vm', maps='vm1')}, providers={'openstack': Provider(name='openstack', supportedGroups=[], providedVMs=['concrete_vm'], storages=[], providedNetworks=['concrete_net'], description=None)}, storages={}, networks=

In [5]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_xmi_model
doml_xmi = parse_xmi_model(doc)

In [6]:
print(dir(doml_xmi))
print(doml_xmi.activeInfrastructure.eClass.eSuperTypes[0].eSuperTypes)

['activeConfiguration', 'activeInfrastructure', 'annotations', 'application', 'concretizations', 'configurations', 'contributesTo', 'description', 'functionalRequirements', 'infrastructure', 'name', 'optimization']
EOrderedSet()


In [7]:
from mc_openapi.doml_mc.intermediate_model.doml_model2im import doml_model_to_im
from mc_openapi.doml_mc.intermediate_model.doml_element import reciprocate_inverse_associations
im = doml_model_to_im(doml_model, mm)
reciprocate_inverse_associations(im, inv_assoc)

In [8]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.doml_element')
pp.pprint(im)

{
    'nginx': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='nginx',
        class_='application_SoftwareComponent',
        attributes={
            'application_SoftwareComponent::isPersistent': False,
            'commons_DOMLElement::name': 'nginx'
        },
        associations={
            'application_SoftwarePackage::consumedInterfaces': set(),
            'application_SoftwarePackage::exposedInterfaces': set()
        }
    ),
    'vm1': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='vm1',
        class_='infrastructure_VirtualMachine',
        attributes={'commons_DOMLElement::name': 'vm1'},
        associations={'infrastructure_ComputingNode::ifaces': set()}
    ),
    'net1': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='net1',
        class_='infrastructure_Network',
        attributes={
            'commons_DOMLElement::name': 'net1',
            'infrastructure_Network::addre

In [9]:
unbound_elems_n = 1
unbound_elems = [f"unbound{i}" for i in range(unbound_elems_n)]

In [10]:
from z3 import Solver

from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel_and_assert_constraints,
    def_attribute_rel_and_assert_constraints,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from mc_openapi.doml_mc.z3encoding.utils import mk_adata_sort

solver = Solver()

class_sort, class_ = mk_class_sort_dict(mm)
assoc_sort, assoc = mk_association_sort_dict(mm)
attr_sort, attr = mk_attribute_sort_dict(mm)
elem_sort, elem = mk_elem_sort_dict(im, unbound_elems)
ss_sort, ss = mk_stringsym_sort_dict(im, mm)
AData = mk_adata_sort(ss_sort)
elem_class_f = def_elem_class_f_and_assert_classes(
    im,
    solver,
    elem_sort,
    elem,
    class_sort,
    class_
)
attr_rel = def_attribute_rel_and_assert_constraints(
    mm,
    solver,
    attr_sort,
    attr,
    class_,
    elem_class_f,
    elem_sort,
    AData,
    ss
)
assert_im_attributes(
    attr_rel,
    solver,
    im,
    mm,
    elem,
    attr_sort,
    attr,
    AData,
    ss
)
assoc_rel = def_association_rel_and_assert_constraints(
    mm,
    solver,
    assoc_sort,
    assoc,
    class_,
    elem_class_f,
    elem_sort,
    inv_assoc
)
assert_im_associations_q(
    assoc_rel,
    solver,
    {k: v for k, v in im.items() if k not in unbound_elems},
    elem,
    assoc_sort,
    assoc,
)

In [11]:
# solver.push()

### Metamodel statistics

In [12]:
from mc_openapi.doml_mc.intermediate_model.metamodel import get_subclasses_dict

print("Number of classes:", len(mm))
print(
    "Number of tuples in the subclass relation:",
    sum(len(s) for s in get_subclasses_dict(mm).values())
)
n_attrs = sum(len(c.attributes) for c in mm.values())
print("Number of attributes:", n_attrs)
print("Number of attribute assertions:", 3 * n_attrs)
n_assocs = sum(len(c.associations) for c in mm.values())
print("Number of associations:", n_assocs)
print("Number of associations assertions:", 3 * n_assocs + len(inv_assoc))



Number of classes: 42
Number of tuples in the subclass relation: 112
Number of attributes: 42
Number of attribute assertions: 126
Number of associations: 53
Number of associations assertions: 165


### DOML model statistics

In [13]:
n_elems = len(im)
print("Number of elements:", n_elems)
n_attrs_im = sum(len(e.attributes) for e in im.values())
print("Number of attributes (IM):", n_attrs_im)
n_assocs_im = sum(len(a) for e in im.values() for a in e.associations.values())
print("Number of associations (IM):", n_assocs_im)
print("Number of string symbols:", len(ss))
print("Number of IM attribute assertions:", n_elems)
print("Number of IM association assertions:", n_elems ** 2)

Number of elements: 8
Number of attributes (IM): 11
Number of associations (IM): 7
Number of string symbols: 10
Number of IM attribute assertions: 8
Number of IM association assertions: 64


In [14]:
solver.check()

sat

In [14]:
solver.statistics()

(:added-eqs               24670
 :arith-make-feasible     7
 :arith-max-columns       7
 :binary-propagations     279
 :conflicts               94
 :datatype-accessor-ax    621
 :datatype-constructor-ax 8501
 :datatype-splits         1752
 :decisions               42376
 :del-clause              282
 :final-checks            8
 :max-generation          4
 :max-memory              23.75
 :memory                  23.75
 :mk-bool-var             2849
 :mk-clause               1085
 :num-allocs              6124957
 :num-checks              1
 :propagations            8373
 :quant-instantiations    591
 :restarts                3
 :rlimit-count            1631237
 :time                    0.26)

In [15]:
from z3 import Consts, ForAll, Exists, Implies, And, Or

In [16]:
# All VMs have a network interface.
vm, iface = Consts(
    "vm iface", elem_sort
)
vmIfaceAssertion = ForAll(
    [vm],
    Implies(
        elem_class_f(vm) == class_["infrastructure_VirtualMachine"],
        Exists(
            [iface],
            And(
                assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], iface)
            )
        )
    )
)
solver.assert_and_track(vmIfaceAssertion, "vm_iface")

In [17]:
solver.check()

sat

In [18]:
solver.statistics().memory

25.28

In [18]:
m = solver.model()
print(m)

[attribute_st_types infrastructure_VirtualMachine::sizeDescription = True,
 attribute_mult_ub infrastructure_Network::address_ub = True,
 attribute_st_types infrastructure_Rule::toPort = True,
 association_st_classes infrastructure_SecurityGroup::ifaces = True,
 associations vm1 concrete_vm = True,
 attribute_st_types application_SaaS::licenseCost = True,
 association_st_classes concrete_ConcreteInfrastructure::storages = True,
 association_st_classes concrete_VirtualMachine::maps = True,
 attribute_mult_ub infrastructure_Storage::cost = True,
 attribute_mult_lb infrastructure_Rule::kind = True,
 attribute_st_types commons_DOMLElement::name = True,
 attribute_st_types infrastructure_Network::address_lb = True,
 association_mult_ub concrete_FunctionAsAService::maps = True,
 associations con_infra con_infra = True,
 attribute_st_types infrastructure_KeyPair::bits = True,
 association_st_classes infrastructure_AutoScalingGroup::securityGroup = True,
 association_inverse infrastructure_Con

In [19]:
from itertools import product
for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()):
    if (e1n in unbound_elems or e2n in unbound_elems) and m.eval(assoc_rel(e1, a, e2)):
        print(e1, a, e2)

vm1 infrastructure_ComputingNode::ifaces unbound0
net1 infrastructure_Network::connectedIfaces unbound0
sg infrastructure_SecurityGroup::ifaces unbound0
unbound0 infrastructure_NetworkInterface::belongsTo net1
unbound0 infrastructure_NetworkInterface::associated sg
